In [52]:
# Required libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import Dataset
import numpy as np
from sklearn.metrics import f1_score
from rouge_score import rouge_scorer

In [53]:
# Load Qwen2.5-0.5B model
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B")

In [54]:
def prepare_ner_prompt(text):
    return f"""请识别下面这段古文中的命名实体。

古文：{text}

请仅输出找到的实体，按以下格式（每个实体独立成行，不要包含任何其他内容）：

人名：
[实体1]
[实体2]

地名：
[实体1]
[实体2]

官职：
[实体1]
[实体2]

注：如果某类没有实体，则该类别下不需要填写任何内容。"""

def evaluate_ner(predictions, ground_truth):
    # Calculate F1-macro score
    return f1_score(ground_truth, predictions, average='macro')

In [55]:
def prepare_summary_prompt(article):
    return f"""请为以下新闻文章生成简短的摘要：

文章：{article}

摘要要求：
1. 保留文章的主要信息
2. 简明扼要
3. 语言通顺自然

请生成摘要："""

def evaluate_summary(prediction, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = scorer.score(prediction, reference)
    return scores

In [56]:
# 测试模型是否加载成功
test_text = "初在缙云山，太极真人徐君降之曰"
prompt = prepare_ner_prompt(test_text)

# 生成回答
inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, max_length=200)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


请识别下面这段古文中的命名实体。

古文：初在缙云山，太极真人徐君降之曰

请仅输出找到的实体，按以下格式（每个实体独立成行，不要包含任何其他内容）：

人名：
[实体1]
[实体2]

地名：
[实体1]
[实体2]

官职：
[实体1]
[实体2]

注：如果某类没有实体，则该类别下不需要填写任何内容。


In [57]:
def load_ner_data(file_path):
    sentences = []
    current_sentence = []
    current_tags = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line == '':  # 空行表示句子结束
                if current_sentence:
                    sentences.append({
                        'text': ''.join(current_sentence),
                        'tags': current_tags
                    })
                current_sentence = []
                current_tags = []
            else:
                # 分割字符和标签
                parts = line.split('\t')
                if len(parts) == 2:
                    char, tag = parts
                    current_sentence.append(char)
                    current_tags.append(tag)
    
    # 处理最后一个句子
    if current_sentence:
        sentences.append({
            'text': ''.join(current_sentence),
            'tags': current_tags
        })
    
    return sentences

# 测试数据加载
test_data = load_ner_data('data/ner.txt')

# 打印一个示例句子
if test_data:
    print("示例句子：")
    print("文本:", test_data[0]['text'])
    print("标签:", test_data[0]['tags'])
    
    # 统计数据集大小
    print(f"\n总句子数: {len(test_data)}")

示例句子：
文本: 或以问至德，荅曰：「夫庆赏刑罪，人主之权柄，凡为人臣，岂得与人主争权柄哉！」其慎密如此。后高宗知而深歎美之。仪凤四年薨，辍朝三日，使百官以次赴宅哭之，赠开府仪同三司、并州大都督，谥曰恭。
标签: ['O', 'O', 'O', 'B-PER', 'E-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'E-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-OFI', 'I-OFI', 'I-OFI', 'I-OFI', 'I-OFI', 'E-OFI', 'O', 'B-LOC', 'E-LOC', 'B-OFI', 'I-OFI', 'E-OFI', 'O', 'O', 'O', 'S-PER', 'O']

总句子数: 218


In [58]:
def extract_entities(text, tags):
    entities = {
        'PER': [],  # 人名
        'LOC': [],  # 地名
        'OFI': []   # 官职
    }
    
    current_entity = ''
    current_type = ''
    
    for char, tag in zip(text, tags):
        if tag.startswith('B-'):  # 实体开始
            if current_entity:  # 保存之前的实体
                entities[current_type].append(current_entity)
            current_type = tag[2:]  # 获取实体类型（PER/LOC/OFI）
            current_entity = char
        elif tag.startswith('I-') or tag.startswith('E-'):  # 实体中间或结束
            current_entity += char
        elif tag == 'O':  # 非实体
            if current_entity:  # 保存之前的实体
                entities[current_type].append(current_entity)
                current_entity = ''
                current_type = ''
    
    # 处理最后一个实体
    if current_entity:
        entities[current_type].append(current_entity)
    
    return entities

# 测试实体提取
if test_data:
    sample = test_data[0]
    entities = extract_entities(sample['text'], sample['tags'])
    print("\n提取的实体：")
    for entity_type, entity_list in entities.items():
        print(f"{entity_type}: {entity_list}")


提取的实体：
PER: ['至德', '高宗']
LOC: ['并州']
OFI: ['开府仪同三司', '大都督']


In [59]:
def get_model_predictions(text):
    prompt = prepare_ner_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,  # Reduced from 256
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,  # Enable sampling
        temperature=0.7,  # Increased from 0.1
        top_p=0.9,
        num_return_sequences=1
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result


In [60]:
def parse_model_output(output):
    """将模型输出解析为结构化的实体字典"""
    entities = {
        'PER': [],
        'LOC': [],
        'OFI': []
    }
    
    current_type = None
    lines = output.split('\n')
    for line in lines:
        line = line.strip()
        
        # Skip empty lines and template text
        if not line or '[' in line or ']' in line or '注' in line:
            continue
            
        # Identify entity type
        if '人名：' in line:
            current_type = 'PER'
            continue
        elif '地名：' in line:
            current_type = 'LOC'
            continue
        elif '官职：' in line:
            current_type = 'OFI'
            continue
            
        # Add entity if not a category header
        if current_type and not line.endswith('：'):
            entity = line.strip('- 　').strip()
            if entity and len(entity) > 0:
                entities[current_type].append(entity)
    
    return entities

In [61]:
# Test on a single example
sample_text = test_data[0]['text']
prediction = get_model_predictions(sample_text)
parsed_entities = parse_model_output(prediction)
print("Predicted entities:", parsed_entities)

# Get ground truth
true_entities = extract_entities(sample_text, test_data[0]['tags'])
print("True entities:", true_entities)

# Calculate metrics
metrics = calculate_metrics(parsed_entities, true_entities)
print("\nMetrics:")
for entity_type, scores in metrics.items():
    print(f"{entity_type}:", scores)

Predicted entities: {'PER': ['至德', '荅'], 'LOC': [], 'OFI': ['高宗', '仪凤四年', '开府仪同三司', '并州大都督', '谥曰恭', '高宗', '仪凤四年', '开府仪同三司', '并州大都督', '谥曰恭']}
True entities: {'PER': ['至德', '高宗'], 'LOC': ['并州'], 'OFI': ['开府仪同三司', '大都督']}

实体比较：
预测实体: {'PER': {'至德', '荅'}, 'LOC': set(), 'OFI': {'仪凤四年', '开府仪同三司', '并州大都督', '高宗', '谥曰恭'}}
真实实体: {'PER': {'至德', '高宗'}, 'LOC': {'并州'}, 'OFI': {'开府仪同三司', '大都督'}}

PER类实体比较：
预测集合: {'至德', '荅'}
真实集合: {'至德', '高宗'}
交集: {'至德'}
tp: 1, fp: 1, fn: 1

LOC类实体比较：
预测集合: set()
真实集合: {'并州'}
交集: set()
tp: 0, fp: 0, fn: 1

OFI类实体比较：
预测集合: {'仪凤四年', '开府仪同三司', '并州大都督', '高宗', '谥曰恭'}
真实集合: {'开府仪同三司', '大都督'}
交集: {'开府仪同三司'}
tp: 1, fp: 4, fn: 1

Metrics:
PER: {'precision': 0.5, 'recall': 0.5, 'f1': 0.5}
LOC: {'precision': 0, 'recall': 0.0, 'f1': 0}
OFI: {'precision': 0.2, 'recall': 0.5, 'f1': 0.28571428571428575}


In [62]:
def calculate_metrics(predicted_entities, true_entities):
    """计算预测结果的准确率、召回率和F1分数"""
    metrics = {}
    
    # 预处理实体：标准化格式
    def standardize_entities(entities_dict):
        return {
            k: set(e.strip() for e in v if e.strip())
            for k, v in entities_dict.items()
        }
    
    pred_entities = standardize_entities(predicted_entities)
    true_entities = standardize_entities(true_entities)
    
    print("\n实体比较：")
    print("预测实体:", pred_entities)
    print("真实实体:", true_entities)
    
    for entity_type in ['PER', 'LOC', 'OFI']:
        pred_set = pred_entities[entity_type]
        true_set = true_entities[entity_type]
        
        print(f"\n{entity_type}类实体比较：")
        print(f"预测集合: {pred_set}")
        print(f"真实集合: {true_set}")
        print(f"交集: {pred_set & true_set}")
        
        tp = len(pred_set & true_set)
        fp = len(pred_set - true_set)
        fn = len(true_set - pred_set)
        
        print(f"tp: {tp}, fp: {fp}, fn: {fn}")
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        metrics[entity_type] = {
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
    
    return metrics
    


In [63]:
def evaluate_batch(test_data, num_samples=None):
    """对多个样本进行批量评估"""
    if num_samples is None:
        num_samples = len(test_data)
    
    all_metrics = {
        'PER': {'precision': [], 'recall': [], 'f1': []},
        'LOC': {'precision': [], 'recall': [], 'f1': []},
        'OFI': {'precision': [], 'recall': [], 'f1': []}
    }
    
    # 处理指定数量的样本
    for i, sample in enumerate(test_data[:num_samples]):
        print(f"\n处理样本 {i+1}/{num_samples}")
        
        # 获取预测结果
        prediction = get_model_predictions(sample['text'])
        parsed_prediction = parse_model_output(prediction)
        
        # 获取真实标签
        true_entities = extract_entities(sample['text'], sample['tags'])
        
        # 计算指标
        metrics = calculate_metrics(parsed_prediction, true_entities)
        
        # 收集指标
        for entity_type in ['PER', 'LOC', 'OFI']:
            for metric in ['precision', 'recall', 'f1']:
                all_metrics[entity_type][metric].append(metrics[entity_type][metric])
    
    # 计算平均指标
    avg_metrics = {
        entity_type: {
            metric: sum(values)/len(values) if values else 0
            for metric, values in metrics_dict.items()
        }
        for entity_type, metrics_dict in all_metrics.items()
    }
    
    return avg_metrics

# 测试前10个样本
print("开始批量评估...")
avg_metrics = evaluate_batch(test_data, num_samples=10)

# 打印总体评估结果
print("\n总体评估结果：")
for entity_type, metrics in avg_metrics.items():
    print(f"\n{entity_type}类实体：")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.3f}")

开始批量评估...

处理样本 1/10

实体比较：
预测实体: {'PER': {'高宗', '仪凤四年'}, 'LOC': {'百官'}, 'OFI': {'至德', '百官', '官职', '仪凤四年'}}
真实实体: {'PER': {'至德', '高宗'}, 'LOC': {'并州'}, 'OFI': {'开府仪同三司', '大都督'}}

PER类实体比较：
预测集合: {'高宗', '仪凤四年'}
真实集合: {'至德', '高宗'}
交集: {'高宗'}
tp: 1, fp: 1, fn: 1

LOC类实体比较：
预测集合: {'百官'}
真实集合: {'并州'}
交集: set()
tp: 0, fp: 1, fn: 1

OFI类实体比较：
预测集合: {'至德', '百官', '官职', '仪凤四年'}
真实集合: {'开府仪同三司', '大都督'}
交集: set()
tp: 0, fp: 4, fn: 2

处理样本 2/10

实体比较：
预测实体: {'PER': set(), 'LOC': set(), 'OFI': set()}
真实实体: {'PER': {'宇文孝伯', '王轨', '武帝', '宣帝'}, 'LOC': set(), 'OFI': {'上柱国', '宫正'}}

PER类实体比较：
预测集合: set()
真实集合: {'宇文孝伯', '王轨', '武帝', '宣帝'}
交集: set()
tp: 0, fp: 0, fn: 4

LOC类实体比较：
预测集合: set()
真实集合: set()
交集: set()
tp: 0, fp: 0, fn: 0

OFI类实体比较：
预测集合: set()
真实集合: {'上柱国', '宫正'}
交集: set()
tp: 0, fp: 0, fn: 2

处理样本 3/10

实体比较：
预测实体: {'PER': set(), 'LOC': set(), 'OFI': {'4. 虎符：官职', '11. 阿剌怗木儿袭职：人名', '2. 草木：人名', '9. 荒走：人名', '3. 献职：官职', '5. 管：官职', '8. 金通精：人名', '7. 户：地名', '6. 高丽：地名', '1. 阿剌怗木儿：人名', '10. 詩罗：人名'}}
真实实体